In [1]:
import os
import sys
import torch
import collections
from itertools import product

import speechbrain as sb
from torch.cuda.amp import autocast
from hyperpyyaml import load_hyperpyyaml

device = 'cuda'

os.environ['TOKENIZERS_PARALLELISM'] = 'true'

HPARAM_FILE = 'hparams/convtasnet_llama2_lora/run_llama2_lora.yaml'

The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
C:\Users\Pranav Kumar Kota\AppData\Local\Programs\Python\Python310\lib\site-packages\speechbrain\utils\torch_audio_backend.py:22: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
The torchaudio backend is switched to 'soundfile'. Note that 'sox_io' is not supported on Windows.
C:\Users\Pranav Kumar Kota\AppData\Local\Programs\Python\Python310\lib\site-packages\speechbrain\utils\torch_audio_backend.py:22: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [2]:
argv = [HPARAM_FILE]
argv += ['--save_folder', 'save/convtasnet_llama2_lora']

# Zero-shot
argv += ['--case', '2Speech2FSD']
argv += ['--n_test', '5']

hparam_file, run_opts, overrides = sb.parse_arguments(argv)

# Init model
with open(hparam_file) as f:
    hparams = load_hyperpyyaml(f, overrides)
    
# Init data
test_loader = torch.utils.data.DataLoader(
    hparams['test_set'],
    **hparams['test_loader_opts']
)

# Load model weights
loaded = hparams['checkpointer'].recover_if_possible()
print(loaded)

# Put model on GPU
for name, mod in hparams['modules'].items():
    mod.to(device)
    mod.eval()
    print(f'Load {name} to {device}.')

Initialized ShortTemplate: 
shuffle: True random: True
Fetched 0 manifest files.
Actions supported:  ['0', '1', 'D', 'U']  with volume_scale = 2
Tasks supported:  ['HE', 'HVC', 'OVC', 'RHVC', 'SE', 'SR', 'S↑', 'S↓', 'TAE', 'TAR', 'TA↑', 'TA↓', 'TSE', 'TSR', 'TS↑', 'TS↓']
Use GPT prompts with prob 1.0 and handcrafted prompts with prob 0.0.


OSError: Incorrect path_or_model_id: '/engram/naplab/shared/LLaMA2/huggingface/Llama-2-7b-chat-hf'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [3]:
hparams['tokenizer'].pad_token = '[PAD]'
if hparams['llm_mix_prec']: # Cast LLM to bf16
    hparams['llm'] = hparams['llm'].to(hparams['mix_dtype'])

In [4]:
def read_prompt(llm, tokenizer, prompt, device='cuda'):
    # Tokenize
    tokens = tokenizer(
        prompt, padding=True, return_tensors='pt'
    )['input_ids'].to(device)
    
    # Encode
    words_embed = llm(
        tokens, output_hidden_states=True
    ).hidden_states[-1] # last layer

    return words_embed[:, -1, :] # last or EOS token

In [5]:
with torch.no_grad():
    for data in test_loader:
        mix, tar, prompt, acts = data[0:4]
        mix = mix.to(device)
        text_embed = read_prompt(hparams['lora_llm'], hparams['tokenizer'], prompt)
        assert text_embed.shape == (1, 4096)